# Export workflow definitions

We need to compose workflows and place the function prototypes into their proper place.

### IMports


In [20]:
from infer_subc_2d.organelles_config.helper import write_workflow_json


-----------------
## WORKFLOWS



-----------------
## NUCLEI workflow

Write the `infer_nuclei` spec to the widget json 

In [39]:
from infer_subc_2d.constants import NUC_CH


def make_infer_nuclei_dict():
    """
    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    step_n = 0
    ###################
    # EXTRACT
    ###################   

    step_name.append(f"{step_n+1}")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = NUC_CH) )
    parent.append(step_n)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    step_n += 1

    ###################
    # PRE_PROCESSING
    ###################                         
    step_name.append(f"{step_n+1}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 4, gauss_sig=1.4 ))
    parent.append(step_n)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    step_n += 1

    ###################
    # CORE_PROCESSING
    ###################
    step_name.append(f"{step_n+1}")
    function_name.append("apply_log_li_threshold")
    category.append("core")
    parameter_values.append(dict(thresh_factor = 0.9, 
                                                            thresh_min = .1,
                                                            thresh_max = 1.))
    parent.append(step_n)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    step_n += 1

    ###################
    # POST_PROCESSING
    ###################
    step_name.append(f"{step_n+1}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 25, 
                                  min_size = 15,
                                 method="slice_by_slice" ))
    parent.append(step_n)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    step_n += 1

    
    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [40]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_nuclei_dict = make_infer_nuclei_dict()

write_workflow_json("conf_1.1.nuclei", infer_nuclei_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc-2D/infer_subc_2d/organelles_config/conf_1.1.nuclei.json')

In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_nuclei_dict = make_infer_nuclei_dict()

write_workflow_json("conf_2.2.nuclei_stepbystep", infer_nuclei_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc-2D/infer_subc_2d/organelles_config/conf_2.2.nuclei_stepbystep.json')

_____________

## SOMA

In [76]:
def make_infer_soma_step_by_step_dict():
    """
    crete .json version of infer_soma
    """
    """
    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    step_n = 1
    ###################
    # EXTRACT
    ###################   
    raw_input_step = 0


    step_name.append(f"{step_n}")
    function_name.append("infer_nuclei")
    category.append("extraction")
    parameter_values.append( dict(
                                nuc_ch = NUC_CH,
                                median_sz = 4,
                                gauss_sig = 1.34,
                                thresh_factor = 0.9,
                                thresh_min = 0.1,
                                thresh_max = 1.0,
                                max_hole_w = 25,
                                small_obj_w = 15)
                                )
                                
    parent.append(raw_input_step)
    annotation.append(f"get  nuclei segmentation: {step_n}")    
    nuc_step = step_n

    step_n += 1                     

    step_name.append(f"{step_n}")
    function_name.append("raw_soma_MCZ")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(raw_input_step)
    annotation.append(f" this creates an aggregate signal for the soma" )
    soma_input_step = step_n

    step_n += 1                     
    ### PRE PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 4, gauss_sig=1.4 ))
    parent.append(soma_input_step)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    struct_img_step = step_n
    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("non_linear_soma_transform_MCZ")
    category.append("preprocessing")
    parameter_values.append(None)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    parent.append(step_n-1)

    step_n += 1

    ### CORE
    step_name.append(f"{step_n}")
    function_name.append("masked_object_thresh")
    category.append("core")
    parameter_values.append(dict( th_method="ave_tri_med",
                                                            cutoff_size = 100,
                                                            th_adjust = 0.5))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")

    step_n += 1

    ### POST_PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 25, 
                                  min_size = 15,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    
    step_n += 1

    ### POST- POST_PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("choose_max_label_soma_union_nucleus")
    category.append("postpostprocessing")
    parameter_values.append(None)
    annotation.append(f"basic soma segmentation: {step_n}")
    parent.append([step_n-1,struct_img_step, nuc_step])
    soma_step = step_n

    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("infer_cytosol")
    category.append("postpostprocessing")
    parameter_values.append(dict(erode_nuclei=True))
    parent.append([nuc_step, soma_step])
    annotation.append(f"basic soma segmentation: {step_n}")
    cyto_step = step_n


    # ###################
    # # EXPORT
    # ###################
    # step_name.append("18")
    # function_name.append("export_PLACE_HOLDER")
    # category.append("export")
    # parameter_values.append(None)
    # parent.append([5, 17])


    ##########################
    out_dict = dict()
    for i,stepn in enumerate(step_name):

        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [77]:

infer_soma_stepbystep_dict = make_infer_soma_step_by_step_dict()
infer_soma_stepbystep_dict

write_workflow_json("conf_1.2.soma", infer_soma_stepbystep_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc-2D/infer_subc_2d/organelles_config/conf_1.2.soma.json')

In [33]:
from infer_subc_2d.constants import NUC_CH

def make_infer_soma_step_by_step_dict2():
    """
    crete .json version of infer_soma
    """
    """
    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    step_n = 1
    ###################
    # EXTRACT
    ###################   
    raw_input_step = 0


    step_name.append(f"{step_n}")
    function_name.append("infer_nuclei")
    category.append("extraction")
    parameter_values.append( dict(
                                nuc_ch = NUC_CH,
                                median_sz = 4,
                                gauss_sig = 1.34,
                                thresh_factor = 0.9,
                                thresh_min = 0.1,
                                thresh_max = 1.0,
                                max_hole_w = 25,
                                small_obj_w = 15)
                                )
                                
    parent.append(raw_input_step)
    annotation.append(f"get  nuclei segmentation: {step_n}")    
    nuc_step = step_n

    step_n += 1                     

    step_name.append(f"{step_n}")
    function_name.append("soma_aggregate")
    category.append("extraction")
    parameter_values.append(dict(w0=0,
                                    w1=6,
                                    w2=0,
                                    w3=2,
                                    w4=0,
                                    w5=1,
                                    w6=0,
                                    w7=0,
                                    w8=0,
                                    w9=0,
                                    scale_min_max = True) 
                )
    parent.append(raw_input_step)
    annotation.append(f" this creates an aggregate signal for the soma" )
    soma_input_step = step_n

    step_n += 1                     
    ### PRE PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("scale_and_smooth")
    category.append("preprocessing")
    parameter_values.append(dict(median_sz = 4, gauss_sig=1.4 ))
    parent.append(soma_input_step)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    struct_img_step = step_n
    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("non_linear_soma_transform_MCZ")
    category.append("preprocessing")
    parameter_values.append(None)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    parent.append(step_n-1)

    step_n += 1

    ### CORE
    step_name.append(f"{step_n}")
    function_name.append("masked_object_thresh")
    category.append("core")
    parameter_values.append(dict( th_method="ave_tri_med",
                                                            cutoff_size = 100,
                                                            th_adjust = 0.5))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")

    step_n += 1

    ### POST_PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("fill_and_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, 
                                 hole_max= 25, 
                                  min_size = 15,
                                 method="slice_by_slice" ))
    parent.append(step_n-1)
    annotation.append(f"basic nuclei segmentation: {step_n}")
    
    step_n += 1

    ### POST- POST_PROCESSING
    step_name.append(f"{step_n}")
    function_name.append("choose_max_label_soma_union_nucleus")
    category.append("postpostprocessing")
    parameter_values.append(None)
    annotation.append(f"basic soma segmentation: {step_n}")
    parent.append([step_n-1,struct_img_step, nuc_step])
    soma_step = step_n

    step_n += 1

    step_name.append(f"{step_n}")
    function_name.append("infer_cytosol")
    category.append("postpostprocessing")
    parameter_values.append(dict(erode_nuclei=True))
    parent.append([nuc_step, soma_step])
    annotation.append(f"basic soma segmentation: {step_n}")
    cyto_step = step_n


    # ###################
    # # EXPORT
    # ###################
    # step_name.append("18")
    # function_name.append("export_PLACE_HOLDER")
    # category.append("export")
    # parameter_values.append(None)
    # parent.append([5, 17])


    ##########################
    out_dict = dict()
    for i,stepn in enumerate(step_name):

        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [34]:

infer_soma_stepbystep_dict2 = make_infer_soma_step_by_step_dict2()

write_workflow_json("conf_1.2.soma2", infer_soma_stepbystep_dict2)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc-2D/infer_subc_2d/organelles_config/conf_1.2.soma2.json')

--------------------------
## CYTOSOL .json


In [70]:
def make_infer_cytosol_dict():
    """
    Procedure to infer cyto from linearly unmixed input. (logical soma AND NOT nucleus)
    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
    annotation = []
    raw_input_step = 0
    ###################
    # EXTRACT
    ###################   
    step_n = 1


    step_name.append(f"{step_n}")
    function_name.append("infer_nuclei")
    category.append("extraction")
    parameter_values.append( dict(
                                nuc_ch = NUC_CH,
                                median_sz = 4,
                                gauss_sig = 1.34,
                                thresh_factor = 0.9,
                                thresh_min = 0.1,
                                thresh_max = 1.0,
                                max_hole_w = 25,
                                small_obj_w = 15)
                                )
                                
    parent.append(raw_input_step)
    annotation.append(f"get  nuclei segmentation: {step_n}")    
    nuc_step = step_n
    step_n += 1                     

    step_name.append(f"{step_n}")
    function_name.append("infer_soma_MCZ")
    category.append("extraction")
    parameter_values.append(dict(
                                                    median_sz = 15,
                                            gauss_sig = 1.34,
                                            mo_method = "ave_tri_med",
                                            mo_adjust = 0.5,
                                            mo_cutoff_size = 150,
                                            max_hole_w = 50,
                                            small_obj_w = 45
                                            ))
    parent.append([raw_input_step, nuc_step])
    annotation.append(f" this creates an aggregate signal for the soma" )
    soma_step = step_n

    step_n += 1                     
    step_name.append(f"{step_n}")
    function_name.append("infer_cytosol")
    category.append("core")
    parameter_values.append(dict(erode_nuclei=True))
    parent.append([nuc_step, soma_step])
    annotation.append(f"basic soma segmentation: {step_n}")
    cyto_step = step_n


    # ###################
    # # EXPORT
    # ###################
    # step_name.append("18")
    # function_name.append("export_PLACE_HOLDER")
    # category.append("export")
    # parameter_values.append(None)
    # parent.append([5, 17])


    ##########################
    out_dict = dict()
    for i,stepn in enumerate(step_name):

        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i],
                            annotation=annotation[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [71]:

infer_cytosol_dict = make_infer_cytosol_dict()

write_workflow_json("conf_1.3.cytosol", infer_cytosol_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc-2D/infer_subc_2d/organelles_config/conf_1.3.cytosol.json')

-------------------------------
## Lysosome

In [ ]:
from infer_subc_2d.constants import LYSO_CH
def make_infer_lysosome_step_by_step_from_raw_dict():
    """
    Procedure to infer lysosome from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # EXTRACT
    ###################   
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # EXTRACT
    ###################   

    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    step_name.append("2")
    function_name.append("fixed_infer_soma")
    category.append("extraction")
    parameter_values.append( None )
    parent.append(1)

    step_name.append("3")
    function_name.append("fixed_infer_nuclei")
    category.append("extraction")
    parameter_values.append( None )
    parent.append([1,2])

    step_name.append("4")
    function_name.append("infer_cytosol")
    category.append("extraction")
    parameter_values.append(dict(erode_nuclei = True ))
    parent.append([2,3])

    step_name.append("5")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = LYSO_CH) )
    parent.append(1)
    
    step_name.append("6")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(5)

    step_name.append("7")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 4 ))
    parent.append(6)

    step_name.append("8")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.34 ))
    parent.append(7)


    step_name.append("9")
    function_name.append("lysosome_spot_filter")
    category.append("core")
    parameter_values.append( None )
    parent.append(8)

    # step_name.append("10")
    # function_name.append("lysosome_filiment_filter")
    # category.append("core")
    # parameter_values.append( None )
    # parent.append(8)
    step_name.append("10")
    function_name.append("filament_filter") 
    category.append("core")
    parameter_values.append( dict( filament_scale=1., filament_cut=.15) )
    parent.append(8)

    step_name.append("11")
    function_name.append("logical_or")
    category.append("core")
    parameter_values.append( None )
    parent.append([9,10])

    step_name.append("12")
    function_name.append("hole_filling")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, hole_max=25**2, fill_2d=True))
    parent.append(11)

    step_name.append("13")
    function_name.append("apply_mask")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([12,4])

    step_name.append("14")
    function_name.append("size_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 3**2  ))
    parent.append(13)

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
    
    return out_dict

In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_lysosome_stepbystep_from_raw_dict = make_infer_lysosome_step_by_step_from_raw_dict()

write_workflow_json("conf_1.4.lysosome_stepbystep_from_raw", infer_lysosome_stepbystep_from_raw_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc-2D/infer_subc_2d/organelles_config/conf_1.4.lysosome_stepbystep_from_raw.json')

-------------------------------
## MITO

In [ ]:
from infer_subc_2d.constants import MITO_CH 

def make_infer_mitochondria_step_by_step_from_raw_dict():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # EXTRACT
    ###################   
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # EXTRACT
    ###################   

    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    step_name.append("2")
    function_name.append("fixed_infer_soma")
    category.append("extraction")
    parameter_values.append( None )
    parent.append(1)

    step_name.append("3")
    function_name.append("fixed_infer_nuclei")
    category.append("extraction")
    parameter_values.append( None )
    parent.append([1,2])

    step_name.append("4")
    function_name.append("infer_cytosol")
    category.append("extraction")
    parameter_values.append(dict(erode_nuclei = True ))
    parent.append([2,3])

    step_name.append("5")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = MITO_CH) )
    parent.append(1)

    
    step_name.append("6")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(5)

    step_name.append("7")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 3 ))
    parent.append(6)

    step_name.append("8")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.34 ))
    parent.append(7)


    step_name.append("9")
    function_name.append("vesselness_slice_by_slice") 
    category.append("core")
    parameter_values.append( dict( sigma=1.5, cutoff=0.05) )
    parent.append(8)


    step_name.append("10")
    function_name.append("apply_mask")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([9,4])

    step_name.append("11")
    function_name.append("size_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 3**2  ))
    parent.append(10)

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
    
    return out_dict



In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_mitochondria_stepbystep_from_raw_dict = make_infer_mitochondria_step_by_step_from_raw_dict()

write_workflow_json("conf_1.5.mitochondria_stepbystep_from_raw", infer_mitochondria_stepbystep_from_raw_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc-2D/infer_subc_2d/organelles_config/conf_1.5.mitochondria_stepbystep_from_raw.json')

-------------------------------
## GOLGI

In [ ]:
from infer_subc_2d.constants import GOLGI_CH
def make_infer_golgi_step_by_step_from_raw_dict():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # EXTRACT
    ###################   
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # EXTRACT
    ###################   

    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    step_name.append("2")
    function_name.append("fixed_infer_soma")
    category.append("extraction")
    parameter_values.append( None )
    parent.append(1)

    step_name.append("3")
    function_name.append("fixed_infer_nuclei")
    category.append("extraction")
    parameter_values.append( None )
    parent.append([1,2])

    step_name.append("4")
    function_name.append("infer_cytosol")
    category.append("extraction")
    parameter_values.append(dict(erode_nuclei = True ))
    parent.append([2,3])

    step_name.append("5")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = GOLGI_CH) )
    parent.append(1)
    
    step_name.append("6")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(5)

    step_name.append("7")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 3 ))
    parent.append(6)

    step_name.append("8")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.34 ))
    parent.append(7)

    step_name.append("9")
    function_name.append("masked_object_thresh") 
    category.append("core")
    parameter_values.append( dict( th_method= "triangle", 
                                                            cutoff_size=1200,
                                                            th_adjust = 0.5) )
    parent.append(8)

    step_name.append("10")
    function_name.append("topology_preserving_thinning") 
    category.append("core")
    parameter_values.append( dict( min_thickness=1.6, thin=1) )
    parent.append(9)

    step_name.append("11")
    function_name.append("spot_filter_slice_by_slice") 
    category.append("core")
    parameter_values.append( dict( log_sigma=1.6, cutoff=0.02) )
    parent.append(9)

    step_name.append("12")
    function_name.append("logical_or")
    category.append("core")
    parameter_values.append( None )
    parent.append([10,11])

    step_name.append("13")
    function_name.append("apply_mask")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([12,4])

    step_name.append("14")
    function_name.append("size_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 3**2  ))
    parent.append(13)

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
    
    return out_dict



In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_golgi_stepbystep_from_raw_dict = make_infer_golgi_step_by_step_from_raw_dict()

write_workflow_json("conf_1.6.golgi_stepbystep_from_raw", infer_golgi_stepbystep_from_raw_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc-2D/infer_subc_2d/organelles_config/conf_1.6.golgi_stepbystep_from_raw.json')

-------------------------------
## PEROXISOME

In [ ]:
def make_infer_peroxisome_step_by_step_from_raw_dict():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # EXTRACT
    ###################   
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # EXTRACT
    ###################   

    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    step_name.append("2")
    function_name.append("fixed_infer_soma")
    category.append("extraction")
    parameter_values.append( None )
    parent.append(1)

    step_name.append("3")
    function_name.append("fixed_infer_nuclei")
    category.append("extraction")
    parameter_values.append( None )
    parent.append([1,2])

    step_name.append("4")
    function_name.append("infer_cytosol")
    category.append("extraction")
    parameter_values.append(dict(erode_nuclei = True ))
    parent.append([2,3])

    step_name.append("5")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = PEROXI_CH) )
    parent.append(1)
    
    step_name.append("6")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(5)

    step_name.append("7")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 3.0 ))
    parent.append(6)

    step_name.append("8")
    function_name.append("spot_filter_slice_by_slice") 
    category.append("core")
    parameter_values.append( dict( log_sigma=1, cutoff=0.01) )
    parent.append(7)

    step_name.append("9")
    function_name.append("apply_mask")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([8,4])

    step_name.append("10")
    function_name.append("size_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 2**2  ))
    parent.append(9)

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
    
    return out_dict



In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_peroxisome_stepbystep_from_raw_dict = make_infer_peroxisome_step_by_step_from_raw_dict()

write_workflow_json("conf_1.7.peroxisome_stepbystep_from_raw", infer_peroxisome_stepbystep_from_raw_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc-2D/infer_subc_2d/organelles_config/conf_1.7.peroxisome_stepbystep_from_raw.json')

-------------------------------
## ER


In [ ]:
from infer_subc_2d.constants import ER_CH
def make_infer_er_step_by_step_from_raw_dict():
    """
    Procedure to infer mitochondria from linearly unmixed input from raw

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # EXTRACT
    ###################   
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    ###################
    # EXTRACT
    ###################   

    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    step_name.append("2")
    function_name.append("fixed_infer_soma")
    category.append("extraction")
    parameter_values.append( None )
    parent.append(1)

    step_name.append("3")
    function_name.append("fixed_infer_nuclei")
    category.append("extraction")
    parameter_values.append( None )
    parent.append([1,2])

    step_name.append("4")
    function_name.append("infer_cytosol")
    category.append("extraction")
    parameter_values.append(dict(erode_nuclei = True ))
    parent.append([2,3])

    step_name.append("5")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = ER_CH) )
    parent.append(1)
    
    step_name.append("6")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(5)

    step_name.append("7")
    function_name.append("edge_preserving_smoothing")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(6)

    step_name.append("8")
    function_name.append("filament_filter") 
    category.append("core")
    parameter_values.append( dict( filament_scale=1., filament_cut=.15) )
    parent.append(7)

    step_name.append("9")
    function_name.append("apply_mask")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([8,4])

    step_name.append("10")
    function_name.append("size_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 2**2  ))
    parent.append(9)

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
    
    return out_dict



In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_er_stepbystep_from_raw_dict = make_infer_er_step_by_step_from_raw_dict()

write_workflow_json("conf_1.8.er_stepbystep_from_raw", infer_er_stepbystep_from_raw_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc-2D/infer_subc_2d/organelles_config/conf_1.8.er_stepbystep_from_raw.json')

-------------------------------
## Write workflow .json
Now that we've added our function specs we can compose workflows.

In [ ]:
from infer_subc_2d.constants import LIPID_CH

def make_infer_lipid_step_by_step_from_raw_dict():
    """
    Procedure to infer lipid from linearly unmixed input.

    Parameters
    ------------
    in_img: np.ndarray
        a 3d image containing all the channels

    soma_mask: np.ndarray
        mask

    Returns
    -------------
    lipid_object
        mask defined extent of NU

    """
    step_name = []
    function_name = []
    category =[]
    parameter_values = []
    parent = []
   
    step_name.append("1")
    function_name.append("fixed_get_optimal_Z_img")
    category.append("extraction")
    parameter_values.append(None)
    parent.append(0)

    step_name.append("2")
    function_name.append("fixed_infer_soma")
    category.append("extraction")
    parameter_values.append( None )
    parent.append(1)

    step_name.append("3")
    function_name.append("fixed_infer_nuclei")
    category.append("extraction")
    parameter_values.append( None )
    parent.append([1,2])

    step_name.append("4")
    function_name.append("infer_cytosol")
    category.append("extraction")
    parameter_values.append(dict(erode_nuclei = True ))
    parent.append([2,3])

    step_name.append("5")
    function_name.append("select_channel_from_raw")
    category.append("extraction")
    parameter_values.append( dict(chan = LIPID_CH) )
    parent.append(1)
    
    step_name.append("6")
    function_name.append("min_max_intensity_normalization")
    category.append("preprocessing")
    parameter_values.append(None)
    parent.append(5)

    step_name.append("7")
    function_name.append("median_filter_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict(size = 4 ))
    parent.append(6)

    step_name.append("8")
    function_name.append("image_smoothing_gaussian_slice_by_slice")
    category.append("preprocessing")
    parameter_values.append(dict( sigma = 1.34 ))
    parent.append(7)

    step_name.append("9")
    function_name.append("apply_threshold")
    category.append("core")
    parameter_values.append(dict(method = "otsu",
                                                            thresh_factor = 0.99, 
                                                            thresh_min = .5,
                                                            thresh_max = 1.))
    parent.append(8)

    step_name.append("10")
    function_name.append("hole_filling")
    category.append("postprocessing")
    parameter_values.append(dict( hole_min=0, hole_max=2.5**2, fill_2d=True))
    parent.append(9)

    step_name.append("11")
    function_name.append("apply_mask")
    category.append("postprocessing")
    parameter_values.append(None)
    parent.append([10,4])

    step_name.append("12")
    function_name.append("size_filter_linear_size")
    category.append("postprocessing")
    parameter_values.append(dict( min_size = 4**2  ))
    parent.append(6)

    out_dict = dict()
    for i,stepn in enumerate(step_name):
        entry = dict(category=category[i],
                            function=function_name[i],
                            parameter_values=parameter_values[i],
                            parent=parent[i]
        )
        if entry['parameter_values'] is None:
            _ = entry.pop('parameter_values')
        out_dict[stepn] = entry
        
    return out_dict

In [ ]:
from infer_subc_2d.organelles_config.helper import write_workflow_json

infer_lipid_stepbystep_from_raw_dict = make_infer_lipid_step_by_step_from_raw_dict()

write_workflow_json("conf_1.9.lipid_stepbystep_from_raw", infer_lipid_stepbystep_from_raw_dict)

PosixPath('/Users/ahenrie/Projects/Imaging/infer-subc-2D/infer_subc_2d/organelles_config/conf_1.9.lipid_stepbystep_from_raw.json')